# 실시간 유추 서비스 만들기

학습시킨 예측 모델은 클라이언트가 새 데이터에서 예측 정보를 가져오는 데 사용할 수 있는 실시간 서비스로 배포할 수 있습니다.

## 작업 영역에 연결

이 Notebook의 작업을 시작하려면 먼저 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 모델 학습 및 등록

이제 모델 학습과 등록을 진행하겠습니다.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## 모델을 웹 서비스로 배포

당뇨 환자일 가능성을 기준으로 환자를 분류하는 기계 학습 모델의 학습과 등록을 완료했습니다. 프로덕션 환경에서는 당뇨 의심 환자만 당뇨 임상 시험 대상으로 지정해야 하는 수술 등에 이 모델을 사용할 수 있습니다. 이 시나리오를 지원하려는 경우 웹 서비스로 모델을 배포합니다.

먼저 작업 영역에 등록한 모델을 확인해 보겠습니다.

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

이제 배포할 모델을 가져옵니다. 기본적으로는 모델 이름을 지정하면 최신 버전이 반환됩니다.

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

이 모델을 호스트하는 웹 서비스를 만들려면 몇 가지 코드와 구성 파일이 필요합니다. 먼저 이러한 항목을 저장할 폴더를 만들겠습니다.

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

모델을 배포하는 웹 서비스에는 입력 데이터를 로드하고, 작업 영역에서 모델을 가져오고, 예측을 생성/반환하기 위한 특정 Python 코드가 필요합니다. 이 코드는 웹 서비스에 배포될 *항목 스크립트*(*채점 스크립트*라고도 함)에 저장됩니다.

스크립트는 두 함수로 구성됩니다.

- **init**: 이 함수는 서비스가 초기화되면 호출되며 일반적으로 모델을 로드하는 데 사용됩니다. 채점 스크립트는 **AZUREML_MODEL_DIR** 환경 변수를 사용하여 모델을 저장할 폴더를 결정합니다.
- **run**: 이 함수는 클라이언트 애플리케이션에서 새 데이터를 제출할 때마다 호출되며 일반적으로 모델의 예측 사항을 유추하는 데 사용됩니다.

In [ ]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

웹 서비스는 컨테이너에서 호스트되며, 이 컨테이너는 초기화 시에 필수 Python 종속성을 설치해야 합니다. 이 경우 채점 스크립트에는 **scikit-learn** 및 채점 웹 서비스에서 사용되는 일부 Azure Machine Learning 전용 패키지가 필요하기 때문에 이러한 항목이 포함된 환경을 만들어 보겠습니다. 그런 다음 해당 환경을 채점 스크립트와 함께 *유추 구성*에 추가하고 환경 및 스크립트가 호스트될 컨테이너에 대한 *배포 구성*을 정의해 보겠습니다.

그러면 이러한 구성을 바탕으로 모델을 서비스로 배포할 수 있습니다.

> **자세한 정보**: 모델 배포 및 대상 실행 환경용 옵션에 대한 자세한 내용은 [설명서](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)를 참조하세요.

배포에서는 컨테이너 이미지를 만드는 프로세스를 먼저 실행한 다음 해당 이미지를 기반으로 웹 서비스를 만드는 프로세스를 실행하므로 시간이 다소 걸릴 수 있습니다. 배포가 정상적으로 완료되면 **정상** 상태가 표시됩니다.

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment(name='service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

배포가 정상적으로 진행되었다면 **정상** 상태를 확인할 수 있습니다. 정상 상태가 표시되지 않으면 다음 코드를 사용하여 서비스 로그를 가져와 문제 해결 시에 참조할 수 있습니다.

In [ ]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

[Azure Machine Learning Studio](https://ml.azure.com)의 작업 영역을 살펴보고 작업 영역에서 배포된 서비스가 표시되는 **엔드포인트** 페이지를 확인합니다.

다음 코드를 실행하여 작업 영역에서 웹 서비스 이름을 검색할 수도 있습니다.

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## 웹 서비스 사용

배포한 서비스는 클라이언트 애플리케이션에서 사용할 수 있습니다.

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

여러 환자를 관찰한 정보를 서비스로 전송한 후 각 환자에 대한 예측을 다시 가져올 수도 있습니다.

In [ ]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

위의 코드는 Azure Machine Learning SDK를 사용하여 컨테이너화된 웹 서비스에 연결한 다음 이 서비스를 사용하여 당뇨병 분류 모델에서 예측을 생성합니다. 프로덕션 환경에서는 Azure Machine Learning SDK를 사용하지 않으며 웹 서비스로의 HTTP 요청만 수행하는 비즈니스 애플리케이션이 모델을 사용할 가능성이 높습니다.

이러한 애플리케이션이 요청을 제출해야 하는 URL을 확인해 보겠습니다.

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

엔드포인트 URI가 확인되면 애플리케이션은 HTTP 요청을 수행하여 JSON 형식으로 환자 데이터를 전송한 다음 예측된 클래스를 다시 수신할 수 있습니다.

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

인증이 필요하지 않은 ACI(Azure Container Instance) 서비스로 웹 서비스를 배포했습니다. 개발 및 테스트 시에는 인증을 사용하지 않아도 되지만, 프로덕션 환경에서는 AKS(Azure Kubernetes Service) 클러스터에 서비스를 배포하고 토큰 기반 인증을 사용하도록 설정하는 것이 좋습니다. 이렇게 하려면 REST 요청에 **인증** 헤더를 포함해야 합니다.

## 서비스 삭제

더 이상 필요하지 않은 서비스는 불필요한 요금이 발생하지 않도록 삭제해야 합니다.

In [ ]:
service.delete()
print ('Service deleted.')

모델을 서비스로 게시하는 과정에 대한 자세한 내용은 [설명서](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)를 참조하세요.